# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 1
inference = not finetuning

# Drive Mount

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# Read Data

In [4]:
import pandas as pd
# data = pd.read_csv('/content/drive/MyDrive/#Research/# GB/train_test_sarcasm_data_mod.csv')
data = pd.read_csv('train_test_sarcasm_data_mod.csv')
data

,male,female,_original_label,test
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1
3,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,0
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0
...,...,...,...,...
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1
9247,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো ভাবী অনেক আগে থেকে ...,0,0
9248,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাট সাক...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাজ্ঞী ...,0,0


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [5]:
temp = data[data['test']!= 1]
test = data[data['test']== 1]

In [6]:
len(temp), len(test)

(7400, 1850)

In [7]:
t1 = temp[['male', '_original_label']]
t2 = temp[['female', '_original_label']]
t2.columns = ['male', '_original_label']
temp = pd.concat([t1, t2], axis = 0)
temp

,male,_original_label
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1
3,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,1
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,1
5,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,1
...,...,...
9240,মিথ্যা কথা । এই স্ত্রীলোক মনে হয় সেরামের কাছ...,0
9241,মিথ্যা কথা । এই মহিলা মনে হয় সেরামের কাছ থেক...,0
9244,শালীর সব পাগলচুদা । মাসরাফি যেমন আমাদের সম্প...,0
9247,ডিজিটাল মিটার চালু হইছে তো ভাবী অনেক আগে থেকে ...,0


In [8]:
# validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=1234))
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.02, random_state=1234))
train = temp.drop(validation.index)

C:\Users\austg\AppData\Local\Temp\ipykernel_18388\2961424500.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.02, random_state=1234))


In [9]:
train = temp.drop(validation.index)
print('Train class distribution: ', train._original_label.value_counts())
diff  = max(train._original_label.value_counts()) - min(train._original_label.value_counts())
print('Difference (Change carefully): ', diff)
ex = train[train._original_label==0].sample(n = diff, random_state = 1234)
train = pd.concat([train, ex])
train = train.reset_index(drop=True)
print('Train class distribution: ', train._original_label.value_counts())

Train class distribution:  _original_label
1    8056
0    6158
Name: count, dtype: int64
Difference (Change carefully):  1898
Train class distribution:  _original_label
1    8056
0    8056
Name: count, dtype: int64


In [10]:
validation

,male,_original_label
9227,সাকিব ভাই আপনি হজ্জ করে আসলেন টাকা তো অনেক কাম...,0
6328,সিনেমাতে ভিলেন । বাস্তবে নায়ক,0
8595,জি বোন আপনি ঠিক বলছেন ! তবে আপনাদের সবাইকে দেশ...,0
7073,আল্লাহ আপনাকে নেক হায়াত দান করুন আর আপনার স্বা...,0
8931,সেদিন নিউমার্কেটে কিছু পুলিশ কর্মকর্তা বাইক চা...,0
...,...,...
3723,মাগো বাপ আর কয়বার প্রোমোট করবি এই মরণরে ? কয়বা...,1
4833,অতীতে আমার আম্মু চিল্লিয়েছে বর্তমানে আমি সগৌরব...,1
3907,আমি সিউর আগের জন্মে হিরোইন অমৃতা আর মাহফুজ ...,1
1846,জুন আন্টি শ্রীময়ী যদি জাল ছিঁড়ে বেরিয়েও যায...,1


In [11]:
test

,male,female,_original_label,test
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1
11,দে বাবা দে বলি দে । জবা গলা জোড়াতে ও পারে,দে মা দে বলি দে । জবা গলা জোড়াতে ও পারে,1,1
20,আপনি কি জন্য আপনার বাপের বয়সী বেডার লগে ক্লাবে...,আপনি কি জন্য আপনার মায়ের বয়সী বেডির লগে ক্লাবে...,1,1
29,বন্ধু মানুষ আসলে বাঙ্গির মতো ।,বন্ধুপত্নী মানুষ আসলে বাঙ্গির মতো ।,1,1
34,আয় হায় ! কি ঢস টা দিলো রে আতিক ভাই !,আয় হায় ! কি ঢস টা দিলো রে বিনিতা বোন !,1,1
...,...,...,...,...
9242,সমানভাবে ভাগ করায় সাংবাদিক সাহেবও খুশি,সমানভাবে ভাগ করায় মহিলা সাংবাদিক সাহেবও খুশি,0,1
9243,ত্রিশ দিন রমজানের পর আজকে শয়তান ছাড়া পেয়েছে ৷,ত্রিশ দিন রমজানের পর আজকে শয়তানি ছাড়া পেয়েছে ৷,0,1
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1


# Load Pretrained Model

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
model_path = "raquiba/sarcasm-detection-BanglaSARC"
tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at raquiba/sarcasm-detection-BanglaSARC and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Pretrained tokens:  32000
Total number of tokens after adding the new ones:  32002


# Dataset Preparation

In [29]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer, train , gender, max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length
        self.train = train
        self.gender = gender

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        if self.train == 1:
            text1 = self.train_csv.iloc[index,0] #0th column all data
        else:
            if self.gender == 'male':
                text1 = self.train_csv.iloc[index,0] #male
            else:
                text1 = self.train_csv.iloc[index,1] #female



        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        try:
            if self.train == 1:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
            else:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)

        except:
          print(index, self.train_csv[index])

BATCH_SIZE = 16
dataset_train= BertDataset(train, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_val= BertDataset( validation, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_test_male = BertDataset(test, tokenizer, train = 0 , gender = 'male', max_length = 512)
dataset_test_female = BertDataset(test, tokenizer, train = 0 , gender = 'female', max_length = 512)

train_loader=DataLoader(dataset=dataset_train,batch_size=BATCH_SIZE, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=BATCH_SIZE, shuffle=False)
test_loader_male=DataLoader(dataset=dataset_test_male,batch_size=BATCH_SIZE, shuffle=False)
test_loader_female=DataLoader(dataset=dataset_test_female,batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# id,am, tid, l = next(iter(train_loader))
# x = b_model(id, am,tid)
# x.hidden_states[0].shape

# Model

In [30]:
class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        # self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)
# loss_fn = nn.BCEWithLogitsLoss()
# #Initialize Optimizer
# optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [ ]:
# id,am, tid, l = next(iter(train_loader))
# x,_ = masked_model(id, am,tid)
# x.shape

# Training Loop Decorator

In [31]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


# Training

In [32]:
SAVE_MODEL = True

from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup
import warnings
warnings.filterwarnings('ignore')

def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            # load_model(network, optimizer, PATH = '/content/drive/MyDrive/#Research/# GB/saved_model/foa_sarcasm.pth')
            load_model(network, optimizer, PATH = 'foa_sarcasm.pth')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            save_model(network, optimizer)
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    # PATH  = rf'/content/drive/MyDrive/#Research/# GB/saved_model/foa_sarcasm.pth'
    PATH  = rf'foa_sarcasm.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return


if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = BATCH_SIZE, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.040417,0.628165,0.037188,0.766892,99.357969,99.358970,0.0001,16,cuda
1,1,2,0.030633,0.804618,0.029137,0.800676,98.938697,201.071659,0.0001,16,cuda
2,1,3,0.021109,0.888654,0.023999,0.851351,98.823960,302.721038,0.0001,16,cuda
3,1,4,0.014159,0.931293,0.020755,0.871622,98.874528,404.406184,0.0001,16,cuda
4,1,5,0.009912,0.953947,0.019361,0.864865,98.862542,506.250943,0.0001,16,cuda
5,1,6,0.007061,0.968409,0.018805,0.854730,98.812679,607.938870,0.0001,16,cuda
6,1,7,0.005110,0.978712,0.018815,0.875000,99.077801,709.873658,0.0001,16,cuda
7,1,8,0.003766,0.984856,0.019223,0.891892,100.110191,812.800656,0.0001,16,cuda
8,1,9,0.002953,0.988394,0.019241,0.888514,100.208915,915.801043,0.0001,16,cuda
9,1,10,0.002347,0.991807,0.019400,0.888514,100.017122,1018.608572,0.0001,16,cuda


Saved model and optimizer at foa_sarcasm.pth
Saved model and optimizer at foa_sarcasm.pth


# Load Model

In [34]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
#   load_model(masked_model, None, PATH = '/content/drive/MyDrive/#Research/# GB/saved_model/foa_sarcasm.pth')
  load_model(masked_model, None, PATH = 'foa_sarcasm.pth')

# Inference

In [35]:

def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()

import warnings
warnings.filterwarnings('ignore')

p, emb_male = get_prediction(test_loader_male, masked_model)
test['pred_male_fod'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader_female, masked_model)
test['pred_female_fod'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_fod']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
x = sum(test['pred_female_fod']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_fod']!=test['pred_female_fod']))

100%|██████████| 116/116 [00:08<00:00, 13.73it/s]

For Male:
Total test data:  1850 Total mismatch:  204 Accuracy:  0.8897297297297297
For Female:
Total test data:  1850 Total mismatch:  206 Accuracy:  0.8886486486486487
Male Female Mismatch:  48


# FOA Result

In [36]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []

    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])

    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['pred_female_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

print('Male Female Mismatch: ', sum(test['pred_male_fod']!=test['pred_female_fod']))

# test.to_csv('/content/drive/MyDrive/#Research/# GB/2,3.revision_foa_sarcasm_result.csv', index = False)
test.to_csv('revision_foa_sarcasm_result.csv', index = False)

Male: 
Accuracy: 0.8897
95% Confidence Interval: (np.float64(0.8753918918918919), np.float64(0.9029864864864865))
mean accuracy: 0.8892972972972973

Female: 
Accuracy: 0.8886
95% Confidence Interval: (np.float64(0.8753918918918919), np.float64(0.9021621621621622))
mean accuracy: 0.8885545945945945
Male Female Mismatch:  48


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('revision_foa_sarcasm_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_fod.values
female_pred = test.pred_female_fod.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.000, 0.009]	SPD Mean: 0.004
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.000, 0.007]	EOD Mean: 0.003


# Mismatched Samples

In [ ]:
test[test['pred_male_fod']!=test['pred_female_fod']]

In [ ]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# Before fine tuning
sum(data['Predicted Value for Original Sentence'] != data['Predicted Value for ner converted'])

218


# Prediction of NER Converted text data

In [ ]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])



dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

100%|██████████| 14/14 [00:02<00:00,  4.99it/s]

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [ ]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [ ]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  36 Accuracy:  0.8348623853211009 Bias:  18


In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/baseline_senti_reSult.csv', index = False)